In [168]:
import pandas as pd


In [169]:
data = pd.read_csv('crowd_data.tsv',sep='\t')

In [170]:
data = data.drop(data[data['LifetimeApprovalRate'].str.rstrip('%').astype('float') / 100.0 < 0.7].index)
data = data.drop(data[data['WorkTimeInSeconds'].astype('float') <20].index)

In [171]:
data.reset_index(drop=True)

,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
2,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
3,2,7QT,Is this triple correct or incorrect?,$0.50,7,2133ICYWE97,Submitted,120,99%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
4,2,7QT,Is this triple correct or incorrect?,$0.50,8,928UJANWZ12,Submitted,60,98%,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,60,9QT,Is this triple correct or incorrect?,$0.50,297,HHCKW1111,Submitted,200,80%,wd:Q21060270,wdt:P27,wd:Q916,1.0,CORRECT,NaN,NaN
179,60,9QT,Is this triple correct or incorrect?,$0.50,298,GGUI83657S,Submitted,120,85%,wd:Q21060270,wdt:P27,wd:Q916,1.0,CORRECT,NaN,NaN
180,61,9QT,Is this triple correct or incorrect?,$0.50,301,AALKMII97,Submitted,240,98%,wd:Q1288004,wdt:P1412,wd:Q13330,2.0,INCORRECT,Object,Q1860
181,61,9QT,Is this triple correct or incorrect?,$0.50,302,HHCKW1111,Submitted,200,80%,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN


In [214]:
"""
Compute the Inter-rater Agreement with Fleiss Kappa.
"""

list = ['7QT','8QT','9QT']
batch_rate = []
for i in list:
    one_batch_ = data.loc[data['HITTypeId'] == i]
    one_batch_ = one_batch_.reset_index(drop=True)

    # Default value
    total_num = one_batch_['HITId'].count()
    p1 = (one_batch_['AnswerID'] == 1).sum()/((total_num))
    p2 = (one_batch_['AnswerID'] == 2).sum()/((total_num))
    p_e = p1**2 + p2**2

    one_batch_['WorkerId'].unique()
    UniqueNames = one_batch_.HITId.unique()
    DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}

    for key in DataFrameDict.keys():
        DataFrameDict[key] = one_batch_[:][one_batch_.HITId == key]
        
    sum_P = 0
    for subjects in DataFrameDict.values():

        sub_num = subjects['HITId'].count()
        co_num = (subjects['AnswerID'] == 1).sum()
        inco_num = (subjects['AnswerID'] == 2).sum()

        P_i = (co_num**2 + inco_num**2 - sub_num)/((sub_num-1)*sub_num)

        sum_P += P_i
        
    P_mean = sum_P/len(DataFrameDict.keys())
    k = round((P_mean - p_e)/(1-p_e),2)

    batch_rate.append(k)

In [216]:
"""
process the data
"""

one_batch_ = data
one_batch_ = one_batch_.reset_index(drop=True)
one_batch_ = one_batch_.drop(columns = ['AnswerLabel'])

UniqueNames = one_batch_.HITId.unique()
DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}

for key in DataFrameDict.keys():
    DataFrameDict[key] = one_batch_[:][one_batch_.HITId == key]

frames = []

for subjects in DataFrameDict.values():

    sub_num = subjects['HITId'].count()
    co_num = (subjects['AnswerID'] == 1).sum()
    inco_num = (subjects['AnswerID'] == 2).sum()

    if co_num < inco_num:
        clean_sub = subjects.dropna()
        
        if len(clean_sub.FixPosition.mode()):
            
            item = clean_sub.FixPosition.mode()[0] 
            if item == '2': 
                item = 'Object'
            
            if clean_sub['FixPosition'].value_counts()[item] >= co_num:

                if item == 'Subject':
                    # print('done')
                    subjects['AnswerID'] =  3 - subjects['AnswerID']
                    subjects.loc[:,'Input1ID'] = 'wd:' + clean_sub[clean_sub['FixPosition'] == item]['FixValue'].iat[0]

                elif item == 'Predicate':
                    # print('done')
                    subjects['AnswerID'] = 3 - subjects['AnswerID']
                    subjects.loc[:,'Input2ID'] = 'wdt:' + clean_sub[clean_sub['FixPosition'] == item]['FixValue'].iat[0]

                elif item == 'Object':
                    # print('done')
                    subjects['AnswerID'] = 3 - subjects['AnswerID']
                    subjects.loc[:,'Input3ID'] = clean_sub[clean_sub['FixPosition'] == item]['FixValue'].iat[0]
    
    frames.append(subjects)
    subjects['support'] = (subjects['AnswerID'] == 1).sum()
    subjects['reject'] = (subjects['AnswerID'] == 2).sum()
one_batch = pd.concat(frames)

for ind in range(len(list)):
   one_batch.loc[one_batch['HITTypeId'] == list[ind],'inter-rate'] = batch_rate[ind]


In [245]:
one_batch = one_batch[['HITId','Input1ID','Input2ID','Input3ID','support','reject','inter-rate']]
one_batch = one_batch.drop_duplicates().reset_index(drop='True')

In [249]:
len(one_batch['Input1ID'].unique())

60

In [247]:
one_batch.to_csv('crowd_opt.csv')

In [264]:
for ind in one_batch.index:
    if one_batch['Input1ID'][ind] == 'wd:P2142':
        print('yes')

yes
